In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Activation, LSTM
from keras.utils import np_utils

import os

# brew install graphviz
# pip3 install graphviz
# pip3 install pydot-ng
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [0]:

# sample sentence
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))  # id -> char ['i', 'l', 'e', 'o', 'h', ...]
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
seq_length = timesteps = 10
num_classes = len(char_set)

In [0]:

dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # char to index
    y = [char_dic[c] for c in y_str]  # char to index

    dataX.append(x)
    dataY.append(y)

# One-hot encoding
dataX = np_utils.to_categorical(dataX, num_classes=num_classes)
# reshape X to be [samples, time steps, features]
dataX = np.reshape(dataX, (-1, seq_length, data_dim))
print(dataX.shape)

# One-hot encoding
dataY = np_utils.to_categorical(dataY, num_classes=num_classes)
# time steps
dataY = np.reshape(dataY, (-1, seq_length, data_dim))
print(dataY.shape)

if you wan -> f you want
f you want ->  you want 
 you want  -> you want t
you want t -> ou want to
ou want to -> u want to 
u want to  ->  want to b
 want to b -> want to bu
want to bu -> ant to bui
ant to bui -> nt to buil
nt to buil -> t to build
t to build ->  to build 
 to build  -> to build a
to build a -> o build a 
o build a  ->  build a s
 build a s -> build a sh
build a sh -> uild a shi
uild a shi -> ild a ship
ild a ship -> ld a ship,
ld a ship, -> d a ship, 
d a ship,  ->  a ship, d
 a ship, d -> a ship, do
a ship, do ->  ship, don
 ship, don -> ship, don'
ship, don' -> hip, don't
hip, don't -> ip, don't 
ip, don't  -> p, don't d
p, don't d -> , don't dr
, don't dr ->  don't dru
 don't dru -> don't drum
don't drum -> on't drum 
on't drum  -> n't drum u
n't drum u -> 't drum up
't drum up -> t drum up 
t drum up  ->  drum up p
 drum up p -> drum up pe
drum up pe -> rum up peo
rum up peo -> um up peop
um up peop -> m up peopl
m up peopl ->  up people
 up people -> up people 


In [0]:

model = Sequential()
model.add(LSTM(num_classes, input_shape=(timesteps, data_dim), return_sequences=True))
model.add(LSTM(num_classes, return_sequences=True))
model.add(TimeDistributed(Dense(num_classes)))

model.add(Activation('softmax'))
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 25)            650       
_________________________________________________________________
activation_1 (Activation)    (None, 10, 25)            0         
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_________________________________________________________________


In [0]:

# Store model graph in png
# (Error occurs on in python interactive shell)
__file__ = "rnn-long-char-keras"
plot_model(model, to_file=os.path.basename(__file__) + '.png', show_shapes=True)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(dataX, dataY, epochs=1000)

predictions = model.predict(dataX, verbose=0)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1000





170/170 [==============================] - 7s 42ms/step - loss: 3.2042 - acc: 0.1312
Epoch 2/1000
170/170 [==============================] - 0s 2ms/step - loss: 3.1654 - acc: 0.1871
Epoch 3/1000
170/170 [==============================] - 0s 2ms/step - loss: 3.1076 - acc: 0.1888
Epoch 4/1000
170/170 [==============================] - 0s 2ms/step - loss: 3.0473 - acc: 0.1894
Epoch 5/1000
170/170 [==============================] - 0s 2ms/step - loss: 3.0123 - acc: 0.1894
Epoch 6/1000
170/170 [==============================] - 0s 2ms/step - loss: 2.9897 - acc: 0.1894
Epoch 7/1000
170/170 [==============================] - 0s 2ms/step - loss: 2.9707 - acc: 0.1894
Epoch 8/1000
170/170 [==============================] - 0s 2ms/step - loss: 2.9537 - acc: 0.1894
Epoch 9/1000
170/170 [==============================] - 0s 2ms/step - loss: 2.9390 - acc: 0.1894
Epoch 10/1000
170/170 

In [0]:
for i, prediction in enumerate(predictions):
    # print(prediction)
    x_index = np.argmax(dataX[i], axis=1)
    x_str = [char_set[j] for j in x_index]

    index = np.argmax(prediction, axis=1)
    result = [char_set[j] for j in index]

    print(''.join(x_str), ' -> ', ''.join(result))

if you wan  ->  l you want
f you want  ->   you want 
 you want   ->  tou want t
you want t  ->   u want to
ou want to  ->    want to 
u want to   ->   want to b
 want to b  ->  tont to bu
want to bu  ->  ont to bui
ant to bui  ->  nd to buil
nt to buil  ->  d to build
t to build  ->  hto build 
 to build   ->  to cuild a
to build a  ->  h build a 
o build a   ->   cuild a s
 build a s  ->  tuild a sh
build a sh  ->  utld a shi
uild a shi  ->   ld a ship
ild a ship  ->  ld a ship,
ld a ship,  ->  e a ship, 
d a ship,   ->   anship, d
 a ship, d  ->  tnship, do
a ship, do  ->  nship, don
 ship, don  ->  thip, don'
ship, don'  ->  iip, don't
hip, don't  ->  ep, don't 
ip, don't   ->  l, don't d
p, don't d  ->  l don't dr
, don't dr  ->   don't dru
 don't dru  ->  ton't drum
don't drum  ->   n't drum 
on't drum   ->   't drum u
n't drum u  ->  dt drum up
't drum up  ->  t arum up 
t drum up   ->  htrum up p
 drum up p  ->  toum up pe
drum up pe  ->   um up peo
rum up peo  ->   m up peop
u